In [1]:
from octis.models.LDA import LDA
from octis.models.NeuralLDA import NeuralLDA
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.preprocessing.preprocessing import Preprocessing

from spacy.lang.el.stop_words import STOP_WORDS as el_stop

In [2]:
import pandas as pd
import string

In [3]:
speeches_df = pd.read_csv('data/data_speeches.csv')
statements_df = pd.read_csv('data/data_statements.csv')

# merge the two datasets
df = pd.concat([speeches_df, statements_df], ignore_index=True)

In [4]:
df.sample(5)

,date,id,url,title,text
743,2023-09-04,32458,https://www.primeminister.gr/2023/09/04/32458,Δήλωση του Πρωθυπουργού Κυριάκου Μητσοτάκη μετ...,"Ευχαριστώ Νίκο και Bibi, ευχαριστώ καταρχάς γι..."
276,2021-06-16,26759,https://www.primeminister.gr/2021/06/16/26759,Δευτερολογία και τριτολογία του Πρωθυπουργού Κ...,"Κύριε Τσίπρα, βρήκα πολύ ενδιαφέρουσα την ιστο..."
1003,2021-09-17,27456,https://www.primeminister.gr/2021/09/17/27456,Δήλωση του Πρωθυπουργού Κυριάκου Μητσοτάκη πρι...,"Κυρίες και κύριοι, Με μεγάλη μου χαρά καλωσορί..."
482,2018-11-22,20837,https://www.primeminister.gr/2018/11/22/20837,“Το δικό μας μνημόνιο είναι με τον ελληνικό λα...,Παρέμβαση στη Βουλή κατά τη συζήτηση του νομοσ...
779,2023-04-22,31735,https://www.primeminister.gr/2023/04/22/31735,Δηλώσεις της Προέδρου της Δημοκρατίας Κατερίνα...,"Κυριάκος Μητσοτάκης: Κυρία Πρόεδρε, έχω την χα..."


In [5]:
# drop date, id, url, title 
df = df.drop(columns=['date', 'id', 'url', 'title'])
df.sample(5)

,text
763,Από τις Βρυξέλλες όπου βρίσκομαι για τη Σύνοδο...
816,Κυριάκος Μητσοτάκης: Καθώς μπαίνουμε πια στον ...
634,"Κυρίες και κύριοι, εκπρόσωποι της Τοπικής Αυ..."
928,Ο Πρωθυπουργός Κυριάκος Μητσοτάκης παραβρέθηκε...
732,Είχαμε μία παραγωγική συζήτηση με τον Πρόεδρο ...


In [6]:
# export the merged dataset
df.to_csv('data/data_merged.csv', index=False)

In [7]:
# Initialize preprocessing
preprocessor = Preprocessing(vocabulary=None, language='greek', 
                             remove_stopwords_spacy=True,
                             lemmatize=True, lowercase=True, 
                             punctuation=string.punctuation, remove_punctuation=True, remove_numbers=True,
                             max_df=0.4, min_df=0.01, min_chars=4,  min_words_docs=20, 
                             verbose=True, num_processes=8)
# preprocess
dataset = preprocessor.preprocess_dataset(documents_path='data/data_merged.csv')

  0%|          | 0/3613 [00:00<?, ?it/s]

created vocab
4253
words filtering done


In [11]:
# save 
dataset.save('data/data_merged_preprocessed')

In [35]:
model = LDA(num_topics=10, chunksize=32, alpha="symmetric",  passes=100)

output = model.train_model(dataset)

print(*list(output.keys()), sep="\n")

topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix


In [36]:
output.keys()

dict_keys(['topic-word-matrix', 'topics', 'topic-document-matrix', 'test-topic-document-matrix'])

In [37]:
for t in output['topics']:
  print(" ".join(t))

έργα έργο οποίο στον έχουν μεγάλη σημαντικό σήμερα ώστε καθώς
έχουν πρέπει οποίο όπως μόνο κάθε μπορεί σήμερα τελικά στους
ολοκλήρωση επένδυση σημαντική στον νέες υψηλής εκπαίδευση εταιρείες παιδιά νέων
θέμα πολιτικό ζήτημα συζήτηση θυμίσω προς θέματα εξωτερική πολιτικού στάση
στις κάθε στον σήμερα όπως χώρα όλες πάντα όλους απέναντι
στις χώρα ευρώ επιχειρήσεις οικονομία προς όπως κατά ανάπτυξη μείωση
ήταν εσείς γιατί αυτά όταν κύριε έχετε ούτε αυτήν διότι
έχουμε οποίο πρέπει κάνουμε μπορούμε οποίες ήταν μπορεί είμαστε νομίζω
πρέπει χώρες στις αφορά επίσης μεταξύ όπως έχουμε συνεργασία προς
χρόνια έχουμε χώρα πρέπει ήταν μετά μέσα στις έχουν μπορεί


In [38]:
npmi = Coherence(texts=dataset.get_corpus(), topk=5, measure='c_npmi')

In [39]:
topic_diversity = TopicDiversity(topk=5)

In [40]:
topic_diversity_score = topic_diversity.score(output)
print("Topic diversity: "+str(topic_diversity_score))

npmi_score = npmi.score(output)
print("Coherence: "+str(npmi_score))

Topic diversity: 0.72
Coherence: 0.020386687524211588


In [41]:
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()

In [42]:
pyLDAvis.lda_model.prepare(model, dataset.get_corpus(), dataset.get_vocabulary())

AttributeError: 'numpy.ndarray' object has no attribute 'get_feature_names_out'